In [1]:
import dataset

In [2]:
import importlib

In [3]:
importlib.reload(dataset)

<module 'dataset' from '/Users/sri/P/Steel/dataset.py'>

In [16]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
import tensorflow as tf

In [6]:
from PIL import Image

In [7]:
root_dir = '/Volumes/Transcend/Data/Steel/'
csv_file = root_dir + 'train.csv'
train_dir = root_dir + 'train/'
mask_dir = root_dir + 'mask/'

In [24]:
img_dataset = dataset.get_dataset(csv_file, train_dir, mask_dir)

100%|██████████| 50272/50272 [00:10<00:00, 4885.20it/s]


ValueError: in converted code:

    /Users/sri/P/Steel/dataset.py:77 get_modified_df  *
        o_df = pd.read_csv(csv_file)
    /anaconda3/envs/tf2/lib/python3.6/site-packages/pandas/io/parsers.py:685 parser_f  *
        return _read(filepath_or_buffer, kwds)
    /anaconda3/envs/tf2/lib/python3.6/site-packages/pandas/io/parsers.py:439 _read  *
        fp_or_buf, _, compression, should_close = get_filepath_or_buffer(
    /anaconda3/envs/tf2/lib/python3.6/site-packages/pandas/io/common.py:224 get_filepath_or_buffer
        raise ValueError(msg.format(_type=type(filepath_or_buffer)))

    ValueError: Invalid file path or buffer object type: <class 'tensorflow.python.framework.ops.Tensor'>


In [8]:
df = pd.read_csv(csv_file)

In [29]:
def gen_masks():
    root_dir = '/Volumes/Transcend/Data/Steel/'
    csv_file = root_dir + 'train.csv'
    train_dir = root_dir + 'train/'
    mask_dir = root_dir + 'mask/'
    df = dataset.get_modified_df(csv_file)
    img_mask = dataset.get_img_paths(df, mask_dir)
    for path in img_mask:
        img = Image.open(path)
        img = np.array(img)
        yield tf.convert_to_tensor(img)

In [ ]:
def gen_imgs():
    root_dir = '/Volumes/Transcend/Data/Steel/'
    csv_file = root_dir + 'train.csv'
    train_dir = root_dir + 'train/'
    mask_dir = root_dir + 'mask/'
    df = dataset.get_modified_df(csv_file)
    img_mask = dataset.get_img_paths(df, train_dir)
    for path in img_mask:
        img = Image.open(path)
        img = np.array(img)
        yield tf.convert_to_tensor(img)

In [33]:
def gen_img_masks():
    root_dir = '/Volumes/Transcend/Data/Steel/'
    csv_file = root_dir + 'train.csv'
    train_dir = root_dir + 'train/'
    mask_dir = root_dir + 'mask/'
    df = dataset.get_modified_df(csv_file)
    img_masks = dataset.get_img_paths(df, mask_dir)
    img_paths = dataset.get_img_paths(df, train_dir)
    for img_path, img_mask in zip(img_paths, img_masks):
        img = Image.open(img_path)
        img = np.array(img)
        mask = Image.open(img_mask)
        mask = np.array(mask)
        yield tf.convert_to_tensor(img), tf.convert_to_tensor(mask)

In [36]:
ds = tf.data.Dataset.from_generator(gen_img_masks, (tf.float32, tf.float32), (tf.TensorShape([256, 1600, 3]), tf.TensorShape([256, 1600, 4])))

In [37]:
count = 0
for img, mask in ds:
    count += 1
    if count > 10:
        break
    print(img.shape)
    print(mask.shape)

100%|██████████| 50272/50272 [00:10<00:00, 4635.41it/s]


(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)
(256, 1600, 3)
(256, 1600, 4)


In [3]:
tf_dataset = dataset.get_dataset(csv_file, train_dir, mask_dir)

100%|██████████| 50272/50272 [00:10<00:00, 4740.41it/s]


In [ ]:
df = dataset.get_modified_df(csv_file)

In [ ]:
all_img_paths = dataset.get_img_paths(df, train_dir)

In [ ]:
all_img_mask_path = dataset.get_img_paths(df, mask_dir)

In [ ]:
image_ds = tf.data.Dataset.from_tensor_slices(all_img_paths)

In [ ]:
image_ds = image_ds.map(dataset.load_process_img)

In [ ]:
mask_ds = tf.data.Dataset.from_tensor_slices(all_img_mask_path)

In [ ]:
mask_ds = mask_ds.map(dataset.load_process_img)

In [ ]:
image_mask_ds = tf.data.Dataset.zip((image_ds, mask_ds))

In [ ]:
'/'.join(all_img_paths[9].split('/')[:-2])

In [ ]:
all_img_masks = dataset.get_img_rle(df)

In [ ]:
mask = dataset.get_img_masks(all_img_masks[9])

In [ ]:
img = Image.fromarray(mask, mode='CMYK')

In [ ]:
img.save(mask_dir + 'test.jpg')

In [ ]:
m_img = Image.open(mask_dir + 'test.jpg')

In [ ]:
import numpy as np

In [ ]:
m_np_img = np.array(m_img)

In [ ]:
np.array_equal(m_np_img, mask)

In [ ]:
def gen_masks(img_paths, img_masks, mask_dir):
    for path, rle in zip(img_paths, img_masks):
        mask = dataset.get_img_masks(rle)
        path = path.split('/')
        img_id = path[-1]
        mask_path = mask_dir + '/' + img_id
        mask_img = Image.fromarray(mask, mode='CMYK')
        mask_img.save(mask_path)

In [ ]:
image_ds = tf.data.Dataset.from_tensor_slices(all_img_paths)

In [ ]:
image_df = image_ds.map(dataset.load_process_img)

In [ ]:
mask_ds = tf.data.Dataset.from_tensor_slices(all_img_masks)

In [ ]:
all_masks = [dataset.get_img_masks(mask) for mask in all_img_masks]

In [ ]:
mask_ds = mask_ds.map(tf.py_function(dataset.get_img_masks))